In [ ]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pyregion
import pyarrow.parquet as pq

In [ ]:
extphot = pq.read_table("/home/bekah/gPhoton2/test_data/e23456/e23456-nd-b00.parquet").to_pandas()

In [ ]:
print(extphot['x'].max())
print(extphot['x'].min())
print(extphot['y'].max())
print(extphot['y'].min())

In [ ]:

filtered_df = extphot[(extphot['x'] >= -20000) & (extphot['x'] <= 20000) & (extphot['y'] >= -20000) & (extphot['y'] <= 20000)]

#df = extphot[(extphot['x'] >= 1500) & (extphot['x'] <= 4000) & (extphot['y'] >= -17000) & (extphot['y'] <= -14000)]

In [ ]:
filtered_df

In [ ]:
plt.scatter(filtered_df['x'],filtered_df['y'],s=0.000001,)

In [ ]:
plt.scatter(filtered_df['x'],filtered_df['y'],c=filtered_df['q'],s=0.00001,)
plt.colorbar()

In [ ]:
hist, x_edges, y_edges = np.histogram2d(filtered_df['x'], filtered_df['y'], bins=1000)

x_bin_indices = np.digitize(filtered_df['x'], x_edges) - 1
y_bin_indices = np.digitize(filtered_df['y'], y_edges) - 1

x_bin_indices[x_bin_indices == len(x_edges) - 1] = len(x_edges) - 2
y_bin_indices[y_bin_indices == len(y_edges) - 1] = len(y_edges) - 2

density_values = hist[x_bin_indices, y_bin_indices]

#filtered_df['density'] = density_values
filtered_df.loc[:, 'density'] = density_values

In [ ]:
plt.scatter(filtered_df['x'],filtered_df['y'],c=filtered_df['density'],s=0.0001,)
plt.colorbar()

In [ ]:
plt.scatter(filtered_df['density'],filtered_df['q'],s=.00001)

In [ ]:
filtered_df.to_csv("filtered_df.csv")

In [ ]:
plt.hexbin(filtered_df['density'],filtered_df['q'])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
axs[0].scatter(filtered_df['x'],filtered_df['y'],c=filtered_df['q'],s=.001,cmap='Greys')
cbar = plt.colorbar(axs[0].collections[0], ax=axs[0])
cbar.set_label('Pulse height (q)')

axs[1].hist2d(filtered_df['x'],filtered_df['y'], bins=200, cmap='Greys')
cbar2 = plt.colorbar(axs[1].collections[0],ax=axs[1])
cbar2.set_label('Density')

plt.tight_layout()

plt.show()

In [ ]:

x_bins = np.linspace(df['x'].min(), df['x'].max(), num=2000)
y_bins = np.linspace(df['y'].min(), df['y'].max(), num=2000)

df['x_bin'] = np.digitize(df['x'], bins=x_bins)
df['y_bin'] = np.digitize(df['y'], bins=y_bins)

bin_stats = df.groupby(['x_bin', 'y_bin']).agg(
    count=('q', 'size'),
    avg_q=('q', 'mean'),
    med_q=('q', 'median'),
    std_q=('q', 'std'),
    min_q=('q','min'),
    max_q=('q','max')
).reset_index()

# Remove bins with no data (count == 0)
bin_stats = bin_stats[bin_stats['count'] > 0]

In [ ]:
grouped_stats = bin_stats.groupby('count')['avg_q'].agg(
    mean_avg_q='mean',
    median_avg_q='median',
    std_avg_q='std'
).reset_index()

In [ ]:
grouped_stats

In [ ]:
plt.plot(grouped_stats['count'],grouped_stats['mean_avg_q'])
plt.scatter(bin_stats['count'],bin_stats['avg_q'],s=0.001,c='grey')
plt.ylabel("mean pulse height (q)")
plt.xlabel("bin counts")

In [ ]:
plt.hist(bin_stats['count'],bins=40)

In [ ]:
df= pd.read_csv("filtered_df.csv")

In [ ]:
df

In [ ]:
filtered_df = df[(df['y'] > 19500) & (df['y'] < 20000)  & (df['x'] > 10000)  & (df['x'] > 12000)]
plt.hist(filtered_df['q'],bins=20)
filtered_df_b = df[(df['y'] > 19500) & (df['y'] < 20000)  & (df['x'] > 8000 )  & (df['x'] < 10000)]
plt.hist(filtered_df_b['q'],bins=20,alpha=.5)
filtered_df_b = df[(df['y'] > 19500) & (df['y'] < 20000)  & (df['x'] > 12000 )  & (df['x'] < 14000)]
plt.hist(filtered_df_b['q'],bins=20,alpha=.5)

In [ ]:
plt.scatter(df['xa'],df['q'],c=df['t'],s=.001)

In [ ]:
plt.hist2d(df['xa'],df['q'],bins=900)